# NDWI composites

For use in stratifying WOfS validation


### Load packages

In [ ]:
%matplotlib inline

import geopandas as gpd
import datacube
from datacube.utils import geometry
# from datacube.helpers import write_geotiff
from datacube.utils.cog import write_cog
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../Scripts')
from deafrica_datahandling import load_ard
from deafrica_dask import create_local_dask_cluster
from deafrica_spatialtools import xr_rasterize
from deafrica_classificationtools import HiddenPrints
from deafrica_plotting import map_shapefile
from deafrica_bandindices import calculate_indices

### Set up a dask cluster

In [ ]:
create_local_dask_cluster(aws_unsigned=False)

### Connect to the datacube

In [ ]:
dc = datacube.Datacube(app='Geomedian_composites')

## Analysis Parameters

In [ ]:
time_range = ('2013','2019')

vector_file = 'data/tiles.shp'
attribute_col = 'id'
aez_region = 'data/AEZs/Western.shp'

products = ['usgs_ls8c_level2_2']
measurements = ['green', 'nir']
resolution = (-30, 30)
output_crs = 'EPSG:6933'
align = (15,15)
dask_chunks= {'x':2500,'y':2500, 'time':1}

export_prefix="results/NDWI_composite/western_NDWI_tile-"

### Set up a query

In [ ]:
# Create a reusable query
query = {'time': time_range,
         'measurements': measurements,
         'resolution': resolution,
         'output_crs': output_crs,
         'align': align,
         'dask_chunks':dask_chunks
         }

query

### Open and clip shapefiles

Open `tiles.shp` and clip it to the boundary shapefile (in this example the `aez_region`)


In [ ]:
#read shapefile
gdf = gpd.read_file(vector_file)

#open shapefile
aez=gpd.read_file(aez_region)

# clip points to region
gdf = gpd.overlay(gdf, aez, how='intersection')

### Add a unique identifier

and plot the tiles using `map_shapefile`, you can hover your mouse over the tiles to see the ID 

In [ ]:
# add an ID column
gdf[attribute_col]=range(0, len(gdf))

#print gdf
# map_shapefile(gdf, attribute_col)

#### Enter a list of tile indexes to load

In [ ]:
# tiles_to_load = [23:]

### Generate a NDWI composite


In [ ]:
%%time

# Loop through polygons in geodataframe and extract satellite data
for index, row in gdf.iloc[gdf['id'].loc[118:]].iterrows():
    
    print("Working on tile: "+str(gdf['id'][index]))
    
    # Get the geometry
    geom = geometry.Geometry(row.geometry.__geo_interface__,
                             geometry.CRS(f'EPSG:{gdf.crs.to_epsg()}'))
    
    # Update dc query with geometry      
    query.update({'geopolygon': geom}) 
    
    with HiddenPrints():
        ds = load_ard(dc=dc, 
                          products=products,                 
                          group_by='solar_day',
                          **query)
    
    # Generate a polygon mask to keep only data within the polygon
#     with HiddenPrints():
#         mask = xr_rasterize(gdf.iloc[[index]], ds)
    
#     # Mask dataset to set pixels outside the polygon to `NaN`
#     ds = ds.where(mask)
    
    with HiddenPrints():
        ds = calculate_indices(ds, 'NDWI',collection='c2', drop=True)
    
    ndwi = ds.NDWI.median('time') 
    
    print("     Writing to file...")    
    write_cog(ndwi.compute(),
             export_prefix+str(row[attribute_col]) + '.tif',
             )
    

---
Code to get geojson from datacube UI 

In [ ]:
# from datacube import Datacube
# from odc.ui import DcViewer
# dc = Datacube()
# ui = DcViewer(dc, "2018", products=['ga_ls8c_gm_2_annual'])
# ui

# x = ui._gui.map.layers[1].data

# from shapely.geometry import shape
# geom = [shape(i) for i in x['features']]
# gdf = gpd.GeoDataFrame({'geometry':geom}, crs='EPSG:4326')
# gdf.to_file('data/tiles.shp')